# [M_04] PROTOKÓŁ: PAMIĘĆ DŁUGOTRWAŁA (QDRANT RAG)

**PROJEKT:** OMNI-OPERATOR-V1  
**SILNIK:** GEMINI 3 FLASH + TEXT-EMBEDDING-004  
**STATUS:** IMPLEMENTACJA_RECOGNITION

Ten moduł odpowiada za budowę "pamięci strategicznej" systemu. Zapisujemy wyniki analiz (M_01) oraz wygenerowane posty (M_02) w wektorowej bazie danych **Qdrant**.

**Dlaczego to robimy?**
1. **Analiza Stylu:** System może sprawdzić, jakie hooki stosował w przeszłości.
2. **Unikanie Powtórzeń:** Agent wie, o czym już pisał, aby nie powielać treści.
3. **Sovereign Data:** Twoja wiedza o tym, co "działa", zostaje na Twoim serwerze w Dockerze.

In [1]:
import os
import sys
import uuid
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from google import genai

# 1. KOREKTA ŚCIEŻKI ROBOCZEJ
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

# Dodanie src do path
sys.path.append(os.path.join(os.getcwd(), "src"))

from src.core.config import settings

# 2. INICJALIZACJA KLIENTÓW
# Qdrant działa na porcie 6333 w Twoim Dockerze
qclient = QdrantClient(url=settings.qdrant_url, timeout=60)

# Konfiguracja Google do Embeddingów (zamiana tekstu na wektory)
client = genai.Client(api_key=settings.gemini_api_key)

print(f"LOG: System pamięci gotowy. Połączono z Qdrant na {settings.qdrant_url}")

LOG: System pamięci gotowy. Połączono z Qdrant na http://localhost:6333


## 1. Tworzenie Kolekcji (Schema bazy wektorowej)

Definiujemy kolekcję `content_memory`. Rozmiar wektora (768) odpowiada najnowszemu modelowi Google `text-embedding-004`.

In [2]:
COLLECTION_NAME = "content_memory"

def init_memory():
    """Tworzy kolekcję w Qdrant, jeśli jeszcze nie istnieje."""
    collections = qclient.get_collections().collections
    exists = any(c.name == COLLECTION_NAME for c in collections)
    
    if not exists:
        print(f"LOG: Tworzę nową kolekcję: {COLLECTION_NAME}")
        qclient.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(size=768, distance=Distance.COSINE),
        )
        print("✅ KOLEKCJA UTWORZONA")
    else:
        print(f"LOG: Kolekcja {COLLECTION_NAME} już istnieje.")

init_memory()

LOG: Kolekcja content_memory już istnieje.


## 2. Funkcja Osadzania (Embedding)

Zamieniamy ludzki tekst na listę liczb (wektor), którą AI potrafi porównywać matematycznie.

In [3]:
def get_embedding(text: str):
    result = client.models.embed_content(
        model="text-embedding-004",
        contents=text
    )
    return result.embeddings[0].values

# TEST:
sample_vec = get_embedding("Inżynieria AI w Kuźni Operatorów")
print(f"LOG: Wygenerowano wektor o długości: {len(sample_vec)}")

LOG: Wygenerowano wektor o długości: 768


## 3. Zapisywanie Kampanii do Pamięci

Implementujemy logikę "zapamiętywania". Zapisujemy treść kampanii wraz z jej metadanymi (temat, platformy).

In [4]:
from datetime import datetime

def save_campaign_to_memory(brief_data: dict, topic: str):
    """Zapisuje raport kampanii do bazy Qdrant, uwzględniając metadane klipów."""
    
    print(f"LOG: Generowanie embeddingu dla tematu: {topic}...")
    # 1. Pobieramy wektor (na podstawie tematu i strategii ogólnej)
    content_to_embed = f"Topic: {topic}. Strategy: {brief_data['overall_strategy']}"
    vector = get_embedding(content_to_embed)
    
    # 2. Przygotowujemy punkt danych
    point_id = str(uuid.uuid4())
    timestamp_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Ekstrakcja metadanych klipów (długości i hooki)
    clips_meta = [
        {
            "idx": c.get('clip_index'),
            "duration": c.get('duration_seconds'),
            "hook_sample": c.get('posts', [{}])[0].get('content', '')[:50]
        } for c in brief_data.get('clip_strategies', [])
    ]
    
    print(f"LOG: Wysyłanie do Qdrant (ID: {point_id}) z metadanymi klipów...")
    
    # 3. Zapis punktu do bazy
    qclient.upsert(
        collection_name=COLLECTION_NAME,
        points=[
            PointStruct(
                id=point_id,
                vector=vector,
                payload={
                    "topic": topic,
                    "strategy": brief_data['overall_strategy'],
                    "clips": clips_meta, # <--- Tu ląduje nasza nowa wiedza o czasie trwania
                    "type": "campaign_brief",
                    "timestamp": timestamp_str
                }
            )
        ]
    )
    print(f"✅ ZAPAMIĘTANO POMYŚLNIE: {topic} ({len(clips_meta)} klipy)")

# TEST ZGODNY Z NOWYM STANDARDEM:
mock_brief = {
    "overall_strategy": "Pozycjonowanie Takzen Dev jako lidera suwerennego AI.",
    "clip_strategies": [
        {"clip_index": 1, "duration_seconds": 25, "posts": [{"content": "Eliminacja SaaS..."}]},
        {"clip_index": 2, "duration_seconds": 45, "posts": [{"content": "Prywatny Docker..."}]}
    ]
}
save_campaign_to_memory(mock_brief, "Suwerenność AI 2026")

LOG: Generowanie embeddingu dla tematu: Suwerenność AI 2026...
LOG: Wysyłanie do Qdrant (ID: ee4acde0-cca7-4220-a12a-d930126bff5d) z metadanymi klipów...
✅ ZAPAMIĘTANO POMYŚLNIE: Suwerenność AI 2026 (2 klipy)


## 4. Wyszukiwanie Semantyczne (RAG Test)

Sprawdzamy, czy system potrafi odnaleźć powiązane treści bez użycia słów kluczowych, bazując na samym "sensie" zapytania.

In [5]:
def search_memory(query: str, limit: int = 2):
    """
    Przeszukuje pamięć przy użyciu nowoczesnego API Qdrant 1.16+.
    """
    print(f"LOG: Generowanie embeddingu dla zapytania: '{query}'...")
    query_vector = get_embedding(query)
    
    print("LOG: Przeszukiwanie bazy wektorowej...")
    
    # Używamy najnowszego API query_points (Standard 2025/2026)
    hits = qclient.query_points(
        collection_name=COLLECTION_NAME,
        query=query_vector,
        limit=limit,
        with_payload=True
    ).points
    
    if not hits:
        print("ℹ️ Brak pasujących wspomnień w bazie.")
        return

    print(f"\n🔎 WYNIKI DLA: '{query}'")
    print("-" * 40)
    for hit in hits:
        score = hit.score
        topic = hit.payload.get('topic', 'Brak tematu')
        strategy = hit.payload.get('strategy', '')[:100]
        print(f" -> [Zgodność: {score:.2f}] {topic}")
        print(f"    Strategia: {strategy}...\n")

# TEST OPERACYJNY:
search_memory("Szukam czegoś o niezależności od SaaS")

LOG: Generowanie embeddingu dla zapytania: 'Szukam czegoś o niezależności od SaaS'...
LOG: Przeszukiwanie bazy wektorowej...

🔎 WYNIKI DLA: 'Szukam czegoś o niezależności od SaaS'
----------------------------------------
 -> [Zgodność: 0.49] Suwerenność AI 2026
    Strategia: Pozycjonowanie Takzen Dev jako lidera suwerennego AI....

 -> [Zgodność: 0.46] Suwerenność AI 2026
    Strategia: Pozycjonowanie Takzen Dev jako lidera suwerennego AI....



## STATUS: MODUŁ 04 ZAKOŃCZONY

Mamy w pełni funkcjonalną pamięć wektorową. Twój system potrafi teraz gromadzić doświadczenie.

**Osiągnięcia:**
1. Integracja lokalnego Qdranta z Embeddingami Google.
2. Możliwość zapisu i odzyskiwania wiedzy (RAG).
3. Podwaliny pod Etap 5 (Dyrygent), gdzie system będzie sprawdzał bazę przed każdym montażem.